In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS silver

In [0]:
%sql
CREATE or replace TABLE silver.sensors (
    sensor_id STRING,
    value DOUBLE,
    pollutant STRING,
    unit STRING,
    datetimeFrom STRING,
    datetimeTo STRING
)

In [0]:
%sql
INSERT INTO silver.sensors

SELECT *
FROM bronze.sensors
EXCEPT
SELECT sensor_id, value, pollutant, unit, datetimeFrom, datetimeTo
FROM silver.sensors

In [0]:
%sql
select count(*)
from silver.sensors


In [0]:
import pandas as pd

sensors = spark.table("silver.sensors")



print("Amount of duplicates:",sensors.toPandas().duplicated().sum())
sensors = sensors.drop_duplicates()
print("Amount of duplicates:",sensors.toPandas().duplicated().sum())

In [0]:
print(sensors.toPandas().isnull().sum())


In [0]:
sensors.write.format("delta").mode("overwrite").saveAsTable("silver.sensors")


In [0]:
%sql
select count(*)
from silver.sensors

In [0]:
%sql
CREATE OR REPLACE TABLE silver.sensors AS
SELECT CAST(sensor_id AS INT) AS sensor_id,
       value,
       pollutant,
       unit,
       cast(datetimeFrom as timestamp) AS datetimeFrom,
       cast(datetimeTo as timestamp) AS datetimeTo
FROM silver.sensors

In [0]:
%sql
ALTER TABLE silver.sensors SET TBLPROPERTIES ('delta.columnMapping.mode' = 'name')

In [0]:
%sql


ALTER TABLE silver.sensors
RENAME COLUMN sensor_id TO id

In [0]:
%sql
select *
from silver.sensors
order by datetimeFrom